# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [229]:
import pandas as pd

def limpeza_de_dados(treinamento, coluna):
    dados_limpos = []
    for e in treinamento[coluna]:
        limpeza = e.replace('\n', '')
        limpeza = limpeza.replace('t.co', '')
        limpeza = limpeza.replace('https://', '')
        limpeza = limpeza.replace('#', '')
        limpeza = limpeza.replace(':', '')
        limpeza = limpeza.replace(',', '')
        limpeza = limpeza.replace('.', '')
        limpeza = limpeza.replace('/', '')
        limpeza = limpeza.replace('//', '')
        limpeza = limpeza.replace('...', '')
        limpeza = limpeza.replace(';', '')
        limpeza = limpeza.replace('\'', '')
        limpeza = limpeza.replace('rt', '')
        limpeza = limpeza.replace('@', '')
        limpeza = limpeza.split(' ')
        dados_limpos.append(limpeza)

    return dados_limpos

def contagem_palavras(dados_limpos):
    contagem = {}
    for e in dados_limpos:
        for all in e:
            if all not in contagem:
                contagem[all] = 1
            else:
                contagem[all] += 1
    del contagem['']
    
    return contagem

def frequencia_das_palavras(contagem):
    probabilidade_palavra = {}
    soma = sum(contagem.values())
    for k in contagem:
        probabilidade_palavra[k] = contagem[k] / soma

    return probabilidade_palavra

def calculador_probabilidade(frases_tratadas, frequencia_palavra_positivo, frequencia_palavra_negativo, probabilidade_positivo_total, probabilidade_negativo_total):
    for e in frases_tratadas: #lista de frases
        resultado = []
        for all in frases_tratadas: #lista de palavras
            conta_positivo = []
            conta_negativo = []
            if all != '': #palavra difrente de ''
                for k  in frequencia_palavra_positivo: #loop pela lista de probabilidades de cada palavra
                    if k == all: #se o nome da frequencia for igual ao nome da palavra
                        conta_positivo.append(frequencia_palavra_positivo[k]) #adicionar frequencia a conta
                for k in frequencia_palavra_negativo:
                    if k == all:
                        conta_negativo.append(frequencia_palavra_negativo[k])
                conta_negativo = np.prod(conta_positivo) #produto da lista de probabilidades de cada palavra da frase
                conta_positivo = np.prod(conta_negativo)
                probabilidade_positivo = conta_negativo * probabilidade_positivo_total #formula naiva bayes
                probabilidade_negativo = conta_positivo * probabilidade_negativo_total
                if probabilidade_positivo > probabilidade_negativo:
                    resultado.append('1')
                else:
                    resultado.append('0')

In [234]:
leitura_excel_treinamento = pd.read_excel('tweets_apple.xlsx', sheet_name='Treinamento')

treinamento_raw_positivo = leitura_excel_treinamento[leitura_excel_treinamento.Relevancia==1]
treinamento_raw_negativo = leitura_excel_treinamento[leitura_excel_treinamento.Relevancia==0]

limpeza_positivo = limpeza_de_dados(treinamento_raw_positivo, 'Treinamento')
limpeza_negativo = limpeza_de_dados(treinamento_raw_negativo, 'Treinamento')

contagem_positivo = contagem_palavras(limpeza_positivo)
contagem_negativo = contagem_palavras(limpeza_negativo)
limpeza_positivo[0]

['',
 'applemusic',
 'mariaimdrunktrvisxx',
 'x',
 'justinbieber',
 'x',
 'youngthug',
 'listen',
 'nowdk92m3zwje']

In [231]:
# P(c) = probabilidade de ser um sim e nao dado o total
# probabilidade de uma palavra ser positiva dado que seja positivo | probabilidade de uma palavra ser negativa dado que seja negativo

probabilidade_positivo_total = len(treinamento_positivo) / len((treinamento_raw_positivo+ treinamento_raw_negativo))
probabilidade_negativo_total = len(treinamento_negativo) / len((treinamento_raw_positivo+ treinamento_raw_negativo))

frequencia_palavra_positivo = frequencia_das_palavras(contagem_positivo)
frequencia_palavra_negativo = frequencia_das_palavras(contagem_negativo)
frequencia_palavra_positivo

{'"paisan': 0.0003990422984836393,
 '$535': 0.0003990422984836393,
 '$spot': 0.0003990422984836393,
 '&amp': 0.02194732641660016,
 '&gt': 0.0003990422984836393,
 '&gt&gt': 0.0003990422984836393,
 '&lt&lt': 0.0003990422984836393,
 '(': 0.0003990422984836393,
 '(+4)oqgfcnhf0y': 0.0003990422984836393,
 '(alphabet)3': 0.0007980845969672786,
 '(amazon)2': 0.0007980845969672786,
 '(apple': 0.0003990422984836393,
 '(facebook)4': 0.0007980845969672786,
 '(off': 0.013966480446927373,
 '(ppqsqeam21)': 0.0003990422984836393,
 '(tiger)mark': 0.0003990422984836393,
 '(unlocked)': 0.0003990422984836393,
 '(wvln7xjp0m)': 0.0003990422984836393,
 '-': 0.0019952114924181963,
 '-cast-': 0.0003990422984836393,
 '-dark': 0.0003990422984836393,
 '-utopia-': 0.0003990422984836393,
 '12': 0.0011971268954509178,
 '128gb': 0.0007980845969672786,
 '12amusa': 0.0003990422984836393,
 '16gb32gb64gb': 0.0003990422984836393,
 '2': 0.0003990422984836393,
 '3': 0.0007980845969672786,
 '3nqbqvvvtw': 0.000399042298483639

In [232]:
leitura_excel_teste = pd.read_excel('tweets_apple.xlsx', sheet_name='Teste')

limpeza_dados = limpeza_de_dados(leitura_excel_teste, 'Teste')
teste = calculador_probabilidade(limpeza_dados, frequencia_palavra_positivo, frequencia_palavra_positivo ,probabilidade_positivo_total, probabilidade_negativo_total)
teste

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
